# SentencePiece

## 1. SentencePiece

- 논문 : https://arxiv.org/pdf/1808.06226.pdf

- 센텐스피스 깃허브 : https://github.com/google/sentencepiece

구글은 BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 SentencePiece를 깃허브에 공개하였습니다.

내부 단어 분리 알고리즘을 사용하기 위해서, 데이터에 단어 토큰화를 먼저 진행한 상태여야 한다면 이 단어 분리 알고리즘을 모든 언어에 사용하는 것은 쉽지 않습니다. 영어와 달리 한국어와 같은 언어는 토큰화부터 쉽지 않기 때문입니다. 

그런데, 이런 사전 토큰화 작업(pretokenization)없이 전처리를 하지 않은 데이터(raw data)에 바로 단어 분리 토크나이저를 사용할 수 있는 토크나이저가 SentencePiece입니다.

## 2. IMDB 리뷰 토큰화로 실습

In [2]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x14e405a0610>)

In [3]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [7]:
print(len(train_df))

50000


In [8]:
# sentencepiece의 입력으로 사용하기 위해 txt파일로 저장
with open('imdb_reveiw.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

- input : 학습시킬 파일

- model_prefix : 만들어질 모델 이름

- vocab_size : 단어 집합의 크기

- model_type : 사용할 모델 (unigram(default), bpe, char, word)

- max_sentence_length: 문장의 최대 길이

- pad_id, pad_piece: pad token id, 값

- unk_id, unk_piece: unknown token id, 값

- bos_id, bos_piece: begin of sentence token id, 값

- eos_id, eos_piece: end of sequence token id, 값

- user_defined_symbols: 사용자 정의 토큰

In [12]:
spm.SentencePieceTrainer.Train('--input=imdb_reveiw.txt --model_prefix=imdb --vocab_size=30000 --model_type=bpe --max_sentence_length=9999')

In [13]:
# .vocab 파일에 학습된 단어들 저장
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
1208,▁Ste,-1205
29752,▁Phel,-29749
6880,TER,-6877
13922,phia,-13919
25324,imps,-25321
22853,Win,-22850
1651,▁teen,-1648
22718,▁Shaggy,-22715
19831,Son,-19828
749,ween,-746


In [14]:
# model 파일을 로드하여 인코딩, 디코딩 작업 가능
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

- encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환

- encode_as_idx : 문장을 입력하면 정수 시퀀스로 변환

In [15]:
lines = [
    "I didn't at all think of it this way.",
    "I have waited a long time for someone to film"
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 29950, 29926, 138, 169, 378, 30, 58, 73, 413, 29945]

I have waited a long time for someone to film
['▁I', '▁have', '▁waited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 9795, 4, 666, 285, 92, 1078, 33, 91]



In [17]:
# 단어 집합의 크기 확인
sp.GetPieceSize()

30000

In [21]:
# 정수 -> 단어
sp.IdToPiece(430)

'▁character'

In [22]:
# 단어 -> 정수
sp.PieceToId('▁character')

430

In [25]:
# 정수 시퀀스 -> 문장
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'I have waited a long time for someone to film'

In [26]:
# 단어 시퀀스 -> 문장
sp.DecodePieces(['▁I', '▁have', '▁waited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

In [27]:
# encode
# out_type = str : 문장 -> pieces
# out_type = int : 문장 -> 정수 시퀀스
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁waited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 9795, 4, 666, 285, 92, 1078, 33, 91]


## 네이버 영화 리뷰 토큰화

In [2]:
import pandas as pd
import sentencepiece as spm
import csv

data_path = 'C:/Users/Myeong/dding/data/딥러닝-자연어처리입문/ratings.txt'

In [3]:
naver_df = pd.read_table(data_path)
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [4]:
naver_df = naver_df.dropna(how='any')
naver_df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [5]:
len(naver_df)

199992

In [6]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [7]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=32000 --model_type=bpe --max_sentence_length=9999')

In [8]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


In [9]:
sp = spm.SentencePieceProcessor()
vocab_file = 'naver.model'
sp.load(vocab_file)

True

In [10]:
lines=[
    '뭐 이딴 것도 영화냐.',
    '진짜 최고의 영화입니다 ㅋㅋ'
]

for line in lines:
    print(line)
    print(sp.EncodeAsIds(line))
    print(sp.EncodeAsPieces(line))
    print()

뭐 이딴 것도 영화냐.
[132, 966, 1296, 2590, 30276]
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']

진짜 최고의 영화입니다 ㅋㅋ
[54, 200, 821, 85]
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']



In [12]:
print(sp.encode(lines[1], out_type=str))
print(sp.encode(lines[1], out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]
